## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-14-02-29-00 +0000,wsj,"President Trump, in a WSJ interview, conveyed ...",https://www.wsj.com/politics/elections/trump-i...
1,2025-12-14-02-28-50 +0000,nyt,What to Know About Brown University,https://www.nytimes.com/2025/12/13/us/brown-un...
2,2025-12-14-02-19-00 +0000,wapo,"At least 2 dead, 8 injured in shooting at Brow...",https://www.washingtonpost.com/nation/2025/12/...
3,2025-12-14-02-08-21 +0000,startribune,Wild President Bill Guerin delighted at trade ...,https://www.startribune.com/minnesota-wild-tra...
4,2025-12-14-02-07-57 +0000,startribune,Can J.J. McCarthy make progress in the Vikings...,https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2378/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,24
102,killed,14
19,university,11
18,brown,8
105,syria,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
45,2025-12-13-21-00-41 +0000,nypost,Trump attends annual Army-Navy game after cond...,https://nypost.com/2025/12/13/us-news/trump-at...,62
58,2025-12-13-19-32-58 +0000,nypost,"Trump tells ISIS, ‘we will retaliate’ after ‘t...",https://nypost.com/2025/12/13/us-news/presiden...,62
14,2025-12-14-00-56-30 +0000,nyt,"3 Americans Killed in ISIS Attack in Syria, Tr...",https://www.nytimes.com/2025/12/13/world/middl...,60
26,2025-12-13-23-10-24 +0000,wapo,Trump vows to retaliate after U.S. troops kill...,https://www.washingtonpost.com/world/2025/12/1...,56
60,2025-12-13-19-22-53 +0000,nypost,Belarus frees 123 prisoners including Nobel Pe...,https://nypost.com/2025/12/13/world-news/belar...,54


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
45,62,2025-12-13-21-00-41 +0000,nypost,Trump attends annual Army-Navy game after cond...,https://nypost.com/2025/12/13/us-news/trump-at...
60,54,2025-12-13-19-22-53 +0000,nypost,Belarus frees 123 prisoners including Nobel Pe...,https://nypost.com/2025/12/13/world-news/belar...
10,43,2025-12-14-01-20-18 +0000,nypost,Terrified Brown University students describe ...,https://nypost.com/2025/12/13/us-news/terrifie...
69,38,2025-12-13-18-37-13 +0000,nypost,Woman stabbed in NYC Macy’s bathroom while cha...,https://nypost.com/2025/12/13/us-news/woman-st...
103,37,2025-12-13-15-26-20 +0000,nypost,Israel kills top Hamas commander Raad Saad wit...,https://nypost.com/2025/12/13/world-news/israe...
59,32,2025-12-13-19-32-50 +0000,nypost,North Korea admits troops hunted mines for Vla...,https://nypost.com/2025/12/13/world-news/north...
105,29,2025-12-13-14-48-43 +0000,cbc,"Organizer arrested after fans rip up seats, th...",https://www.cbc.ca/news/world/soccer-fans-mess...
0,26,2025-12-14-02-29-00 +0000,wsj,"President Trump, in a WSJ interview, conveyed ...",https://www.wsj.com/politics/elections/trump-i...
119,25,2025-12-13-12-41-13 +0000,nypost,Outgoing NYC Mayor Eric Adams warns ‘dark days...,https://nypost.com/2025/12/13/us-news/outgoing...
33,23,2025-12-13-22-19-55 +0000,nypost,Engine failure forces Tokyo-bound United Airli...,https://nypost.com/2025/12/13/us-news/engine-f...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
